# Additional parsing of the dataset

Here, we created the script where we additionally parsed data and combined training13b.json and retrived_articles_sampled.json file
We removed data which may not be relevant for eaxtraction

In [1]:
import json
import random


# retreived_articles_sampled.json path
#retreived_articles_sampled_path = "../datasets/training/retrieved_articles_sampled.json"
#retreived_articles_sampled_path = "../datasets/test_golden_answers/batch_1/retrieved_articles_sampled_test_batch_1.json"
#retreived_articles_sampled_path = "../datasets/test_golden_answers/batch_2/retreived_articles_sampled_test_batch_2.json"
#retreived_articles_sampled_path = "../datasets/test_golden_answers/batch_3/retreived_articles_sampled_test_batch_3.json"
retreived_articles_sampled_path = "../datasets/test_golden_answers/batch_4/retreived_articles_sampled_test_batch_4.json"

# path to the original file which is downloaded form the BioASQ site path
#original_downloaded_file_path = '../datasets/training/training13b.json'
#original_downloaded_file_path = '../datasets/test_golden_answers/batch_1/BioASQ-task13bPhaseB-testset1'
#original_downloaded_file_path = '../datasets/test_golden_answers/batch_2/BioASQ-task13bPhaseB-testset2'
#original_downloaded_file_path = '../datasets/test_golden_answers/batch_3/BioASQ-task13bPhaseB-testset3'
original_downloaded_file_path = '../datasets/test_golden_answers/batch_4/BioASQ-task13bPhaseB-testset4'

# link to parsed json file
#parsed_data_path = "../datasets/training/parsed_data_final.json"
#parsed_data_path = "../datasets/test_golden_answers/batch_1/parsed_data_final_test_batch_1.json"
#parsed_data_path = "../datasets/test_golden_answers/batch_2/parsed_data_final_test_batch_2.json"
#parsed_data_path = "../datasets/test_golden_answers/batch_3/parsed_data_final_test_batch_3.json"
parsed_data_path = "../datasets/test_golden_answers/batch_4/parsed_data_final_test_batch_4.json"

In [2]:
# Load sample document dataset
with open(retreived_articles_sampled_path, 'r', encoding='utf-8') as f:
    retreived_articles_sampled_data = json.load(f)['results']  # assumes your sample is under the 'results' key

# Load BioASQ training dataset
with open(original_downloaded_file_path, 'r', encoding='utf-8') as f:
    training13b_data = json.load(f)['questions']  

In [3]:
# this is function that converts the PubMed ID to a link, as same as showed in the training13b.json
def convert_pid_to_link(pid):
    """
    Convert PubMed ID to link.
    """
    link_prefix = "http://www.ncbi.nlm.nih.gov/pubmed/"
    return link_prefix + pid if pid and link_prefix not in pid else pid

Now, we will extract relevant info from each dataset
retreived_articles_sampled.json -> [question id, question, error_rate, ground truth list, articles (with info)]
training_articles_sampled.json -> [question id, ground truth snippets]

We will first create dictionary from training13b.json, because we need only groun truth snippets


In [4]:
training13b_gt_snippets = {}
for element in training13b_data:
    training13b_gt_snippets[element['id']] = element['snippets']

In [5]:
# extract the text from the retrieved articles
retreived_articles_sampled_relevant_info = []
for object in retreived_articles_sampled_data:
    # we need for each article to convert pid from the id to link
    relevant_articles = []
    for article in object['details']:
        article['pid'] = convert_pid_to_link(article['pid'])
        relevant_articles.append(article)
    

    # randomize elements in articles
    random.shuffle(relevant_articles)


    relevant_element = {
        'qid': object['qid'],
        'question': object['question'],
        'ground_truth_documents_pid': [convert_pid_to_link(gt_pid) for gt_pid in object['ground_truth']], 
        'error_rate': object['error_rate'],
        'ground_truth_snippets': training13b_gt_snippets[object['qid']],
        'all_retreived_articles': relevant_articles
    }
    retreived_articles_sampled_relevant_info.append(relevant_element)

print(retreived_articles_sampled_relevant_info[0:3])

[{'qid': '67e6cf2618b1e36f2e0000d0', 'question': 'Should Zotiraciclib be used for glioblastoma?', 'ground_truth_documents_pid': ['http://www.ncbi.nlm.nih.gov/pubmed/38689623', 'http://www.ncbi.nlm.nih.gov/pubmed/38159337'], 'error_rate': {'value': 1.0, 'details': '2 found out of 2'}, 'ground_truth_snippets': [{'beginSection': 'abstract', 'endSection': 'abstract', 'text': '.CONCLUSIONS: TG02 exhibits overlapping toxicity with alkylating agents and low single agent clinical activity in recurrent glioblastoma. ', 'document': 'http://www.ncbi.nlm.nih.gov/pubmed/38159337', 'offsetInBeginSection': 1538, 'offsetInEndSection': 1676}, {'beginSection': 'abstract', 'endSection': 'abstract', 'text': 'Zotiraciclib, a CDK9 inhibitor, has demonstrated efficacy in glioblastoma treatment in preclinical and clinical studies, showing its potential as a therapeutic breakthrough. ', 'document': 'http://www.ncbi.nlm.nih.gov/pubmed/38689623', 'offsetInBeginSection': 498, 'offsetInEndSection': 672}], 'all_ret

In [6]:
# write this info to the json file
parsed_data_final = {
    'data': retreived_articles_sampled_relevant_info
}

data_serialized = json.dumps(parsed_data_final, indent = 4)

with open(parsed_data_path, 'w') as outfile:
    outfile.write(data_serialized)
